In [1]:
!pip install DeepFace

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.9 MB 262.1 kB/s eta 0:00:06
   ----------- ---------------------------- 0.5/1.9 MB 262.1 kB/s eta 0:00:06
   ----------- ---------------------------- 0.5/1.9 MB 262.1 kB/s eta 0:00:06
   ----------- ---------------------------- 0.5/1.9 MB 262.1 kB/s eta 0:00:06
   ---------------- ----------------------- 0.8/1.9 MB 294.4 kB/s eta 0:00:04
   ----------------------


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from deepface import DeepFace
import cv2
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Analyze frame for emotion detection
    analysis = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)

    # Extract and display emotion
    dominant_emotion = analysis[0]['dominant_emotion']
    print("Detected Emotion:", dominant_emotion)

    cv2.putText(frame, dominant_emotion, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

25-02-19 12:38:57 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: C:\Users\yvarajan\.deepface\weights\facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:08<00:00, 701kB/s]


Detected Emotion: angry
Detected Emotion: neutral
Detected Emotion: surprise
Detected Emotion: happy
Detected Emotion: surprise
Detected Emotion: surprise
Detected Emotion: happy
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: neutral
Detected Emotion: happy
Detected Emotion: neutral
Detected Emotion: happy
Detected Emotion: neutral
Detected Emotion: happy
Detected Emotion: happy
Detected Emotion: happy
Detected Emotion: happy
Detected Emotion: neutral
Detected Emotion: surprise
Detected Emotion: surprise
Detected Emotion: surprise
Detected Emotion: surprise
Detected Emotion: surprise
Detected Emotion: neutral
Detected Emotion: surprise
Detected Emotion: angry
Detected Emotion: neutral
Detected Emotion: neutral
Detected Emotion: angry
Detected Emotion: neutral
Detected Emotion: neutral
Detected Emotion: happy
Detected Emotion: happy
Detected Emotion: happy
Detected Emotion: fear
Detected Emotion: fear
Detected Emotion: neutr

In [ ]:
from fastapi import FastAPI, WebSocket
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from fastapi.requests import Request
import cv2
import base64
import uvicorn
app = FastAPI()

# Mount static files for styles, scripts, etc.
app.mount("/static", StaticFiles(directory="static"), name="static")

# Set up Jinja2 templates
templates = Jinja2Templates(directory="templates")


@app.get("/", response_class=HTMLResponse)
async def get_index(request: Request):
    """Serve the index.html file."""
    return templates.TemplateResponse("index.html", {"request": request})


@app.websocket("/stream")
async def stream(websocket: WebSocket):
    """Handle WebSocket connections to stream video frames."""
    await websocket.accept()

    # Open the default camera
    cap = cv2.VideoCapture(0)

    try:
        while True:
            # Read a frame from the camera
            ret, frame = cap.read()
            if not ret:
                break

            # Encode the frame as JPEG
            _, buffer = cv2.imencode(".jpg", frame)

            # Convert the frame to base64
            frame_data = base64.b64encode(buffer).decode("utf-8")

            # Send the frame to the client
            await websocket.send_text(frame_data)
    except Exception as e:
        print(f"Error: {e}")
    finally:
        cap.release()
        await websocket.close()
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)